In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

In [ ]:
import tensorflow as tf

In [ ]:
sess = tf.InteractiveSession #式の作成と実行を交互に行うのでこれを指定する

In [ ]:
#器の用意
x = tf.placeholder(tf.float32, shape = [None,784])
y_ = tf.placeholder(tf.float32, shape = [None,10])

In [ ]:
#重み。行列を渡すと初期化して返す
def weight_variable(shape):
    #truncated_normal:正規分布の左右を切り取ったもののイメージ
    #stddev:標準偏差
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

In [ ]:
#バイアス
def bias_variable(shape):
    #0だと計算が進まないのでバイアス値は少し
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
#たたみ込み処理の定義
def conv2d(x, W):
    #strides = [1,1,1,1]:1pix毎に処理する事
    #padding = 'SAME':左右に0を足したものにする
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding = 'SAME')

In [ ]:
#たたみ込んだ画像をsizeを小さくして特徴を抽出。一番sizeの大きなデータをかえす
def max_pool_2x2(x):
    #ksize適用数範囲2*2
    #strides 2*2ずらして処理
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [ ]:
#1層目の重みの初期化　５＊５の画像に１このデータが３２枚
W_conv1 = weight_variable([5,5,1,32])

In [ ]:
#bias
b_conv1 = bias_variable([32])

In [ ]:
#入力画像の格納。-1は実行時に自動的に値を設定してくれるらしい
x_image = tf.reshape(x, [-1,28,28,1])

In [ ]:
#フィルタをかけた値にReLUの変換をした値
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)

In [ ]:
#プーリング。次元圧縮。今回はMax poolingを採用
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
#2段目
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
#一層目の値を上の値をReLUにかける
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
#プーリング
h_pool1 = max_pool_2x2(h_conv2)